### A logistic regression forward pass 

In [41]:
import torch.nn.functional as F
import torch
from torch.autograd import grad

In [42]:
y= torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0] , requires_grad=True)

In [43]:
# compute in an output layer
z = x1*w1+b
a = torch.sigmoid(z)

In [44]:
a, z

(tensor([0.9183], grad_fn=<SigmoidBackward0>),
 tensor([2.4200], grad_fn=<AddBackward0>))

In [45]:
loss = F.binary_cross_entropy(a,y)

In [46]:
loss

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)

### Updating w1 and b

In [47]:
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

In [48]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


### Multilayer perceptron with two hidden layers

In [49]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs:int, num_output:int):
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layers
            torch.nn.Linear(num_inputs,30), 
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),

            # Output layer 
            torch.nn.Linear(20, num_output)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [50]:
# Instantiation of new NeuralNetwork model
model = NeuralNetwork(num_inputs=50, num_output=3)

In [51]:
# Printing our model summary 
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [52]:
# Checking for total number of trainable parameters of our model 
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad==True)
print("Total number of trainable parametres are: ", num_params)

Total number of trainable parametres are:  2213


In [53]:
# we can access the corresponding weight parameter matrix(of a specifique layer) as follows
print(model.layers[0].weight[0])

tensor([-0.0281,  0.1284,  0.1009, -0.0112, -0.0781, -0.0553, -0.0560,  0.0784,
        -0.0010, -0.0326, -0.1146,  0.0116,  0.0874,  0.0893,  0.0122,  0.1300,
        -0.0350,  0.1099,  0.0790,  0.1249,  0.0729, -0.1273,  0.0692, -0.1022,
        -0.0947, -0.0993, -0.0655, -0.1270,  0.1208, -0.0596,  0.0926,  0.1268,
         0.0897,  0.1052,  0.0036, -0.1390,  0.0863, -0.1192,  0.0366, -0.0590,
         0.0906,  0.0944, -0.0074, -0.0493,  0.1094, -0.0445, -0.1089, -0.0914,
        -0.1354,  0.1046], grad_fn=<SelectBackward0>)


In [54]:
print(model.layers[0].bias)

Parameter containing:
tensor([ 0.0560, -0.0131,  0.1141,  0.0025,  0.0788,  0.1260,  0.0459,  0.0039,
        -0.0099, -0.1152, -0.1105,  0.0472,  0.0131,  0.0366, -0.1277,  0.0792,
        -0.0146,  0.0551, -0.0780,  0.0295,  0.0728, -0.1057,  0.0126, -0.0490,
         0.1161, -0.0296, -0.0827, -0.0906, -0.0129,  0.0642],
       requires_grad=True)


In [55]:
# Generate random training data 
X = torch.rand((1,50))
X

tensor([[0.4049, 0.7199, 0.1914, 0.1110, 0.2571, 0.9903, 0.5465, 0.6608, 0.7740,
         0.4193, 0.8244, 0.0844, 0.2703, 0.9241, 0.3095, 0.1921, 0.0173, 0.0214,
         0.5597, 0.1463, 0.0092, 0.9721, 0.6788, 0.8706, 0.6227, 0.4121, 0.1727,
         0.0201, 0.0579, 0.0127, 0.4383, 0.5526, 0.9336, 0.4040, 0.0474, 0.1752,
         0.9438, 0.8593, 0.6749, 0.1563, 0.5821, 0.2843, 0.7466, 0.6320, 0.8265,
         0.4360, 0.9963, 0.6321, 0.8896, 0.5159]])

In [56]:
# training the model (Execute forward pass of a model)
out = model(X)
out

tensor([[0.1656, 0.0817, 0.0549]], grad_fn=<AddmmBackward0>)

In [57]:
# This tells PyTorch that it doesn't need to keep track of the
# gradients, which can result in significant savings in memory and
# computation.

with torch.no_grad():
    out = model(X)
print(out)

tensor([[0.1656, 0.0817, 0.0549]])


In [58]:
# Using softmax fonction as the activation function 
with torch.no_grad():
    out =torch.softmax(model(X),dim=1)
print(out)

tensor([[0.3553, 0.3267, 0.3180]])


### Setting up efficient data loaders

In [59]:
# Creating a small toy dataset

# Train data 
X_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5],
])

y_train = torch.tensor([0,0,0,1,1])

# test data
X_test = torch.tensor([
    [-0.8,2.8],
    [2.6,-1.6],
    ])

y_test = torch.tensor([0,1])

This Custom ToyDataset class's purpose is to use it to instantiate a pytorch DataLoader.

In [60]:
# creating the custom Dataset class
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X , y):
        self.feature = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.feature[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]

In [61]:
train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test, y_test)

In [62]:
# Instantiating data loaders 
from torch.utils.data import DataLoader

torch.manual_seed(123)

# train DataLoader
train_loader = DataLoader(
    dataset=train_ds,
    batch_size= 2,
    shuffle=True,
    num_workers=0
)

# Test DataLoader 
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [63]:
# iterate over the train_loader
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x , y)
    print("\n")

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])




This is desired to prevent deep neural networks getting caught in repetitive update
cycles during training. To prevent this, it's recommended to set drop_last=True, which
will drop the last batch in each epoch, as shown below:

In [64]:
# train DataLoader
train_loader = DataLoader(
    dataset=train_ds,
    batch_size= 2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [65]:
# iterate once more over the train_loader
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x , y)
    print("\n")

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])




Training Loop

In [66]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_output=2)
optimizer = torch.optim.SGD(model.parameters(),lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx,(features,labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1}/{num_epochs} | Batch: {batch_idx}/ {len(train_loader)} | train Loss: {loss:.2f}")
    model.eval()
    

Epoch: 1/3 | Batch: 0/ 2 | train Loss: 0.75
Epoch: 1/3 | Batch: 1/ 2 | train Loss: 0.65
Epoch: 2/3 | Batch: 0/ 2 | train Loss: 0.44
Epoch: 2/3 | Batch: 1/ 2 | train Loss: 0.13
Epoch: 3/3 | Batch: 0/ 2 | train Loss: 0.03
Epoch: 3/3 | Batch: 1/ 2 | train Loss: 0.00


Model evaluation

In [67]:
with torch.no_grad():
        outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


Using softmax to obtain the class membership probabilities

In [73]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(input=outputs, dim=1, dtype=float)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]], dtype=torch.float64)


We can convert these values into class labels predictions using PyTorch's
argmax function, which returns the index position of the highest value in each
row if we set dim=1 (setting dim=0 would return the highest value in each
column, instead):

In [74]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


Note that it is unnecessary to compute softmax probabilities to obtain the
class labels. We could also apply the argmax function to the logits (outputs)
directly

In [75]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


comparing predict values to real values 

In [76]:
predictions == y_train

tensor([True, True, True, True, True])

compute the prediction accuracy 

In [82]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0 
    total_exemple = 0

    for idx, (features,labels) in enumerate(dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_exemple += len(compare)

    return (correct / total_exemple)


accuracy sur l'ensemble d'entrainement

In [83]:
compute_accuracy(model= model, dataloader = train_loader)

tensor(1.)

accuracy de l'ensemble de test

In [84]:
compute_accuracy(model = model, dataloader = test_loader)

tensor(1.)

Saving an loading models

In [85]:
# recommanded way to how to save a model with pytorch 
torch.save(model.state_dict(), "model.pth")

The model's state_dict is a Python dictionary object that maps each layer in
the model to its trainable parameters (weights and biases). Note that
"model.pth" is an arbitrary filename for the model file saved to disk. We can
give it any name and file ending we like; however, .pth and .pt are the most
common conventions

Once we saved the model, we can restore it from disk as follows:


In [ ]:
model = NeuralNetwork(2,2)

model.load_state_dict(torch.load(f="model.pth"))

The torch.load("model.pth") function reads the file "model.pth" and
reconstructs the Python dictionary object containing the model's parameters
while model.load_state_dict() applies these parameters to the model,
effectively restoring its learned state from when we saved it.


Note that the line model = NeuralNetwork(2, 2) above is not strictly
necessary if you execute this code in the same session where you saved a
model. However, I included it here to illustrate that we need an instance of
the model in memory to apply the saved parameters. Here, the
NeuralNetwork(2, 2) architecture needs to match the original saved model
exactly